## Multi-Agent Customer Support System using Local LLM (DeepSeek-R1 via Ollama)

## 0. Prerequisites

Before running this notebook, make sure you have:

1. **Python 3.10+**
2. **Ollama installed on your machine**  
   - Download from: https://ollama.com/download
3. **DeepSeek model pulled in Ollama**, e.g.:

```bash
ollama pull deepseek-r1:8b

In [1]:
import requests

def test_local_llm(model: str = "deepseek-r1:8b"):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": "Say a short hello.",
        "stream": False,
    }
    resp = requests.post(url, json=payload)
    resp.raise_for_status()
    data = resp.json()
    print("LLM raw response:", data.get("response", "")[:200])

test_local_llm()

LLM raw response: Hello! 😊 How can I assist you today?


In [2]:
from agents.data_agent import CustomerDataAgent
from agents.support_agent import SupportAgentLLM
from agents.router_agent import RouterAgentLLM

In [3]:
data_agent = CustomerDataAgent()
support_agent = SupportAgentLLM(data_agent)
router = RouterAgentLLM(data_agent, support_agent)

print("Agents initialized:")
print(" - data_agent:", type(data_agent))
print(" - support_agent:", type(support_agent))
print(" - router:", type(router))

Agents initialized:
 - data_agent: <class 'agents.data_agent.CustomerDataAgent'>
 - support_agent: <class 'agents.support_agent.SupportAgentLLM'>
 - router: <class 'agents.router_agent.RouterAgentLLM'>


In [4]:
def run_scenario(query: str):
    print("=" * 80)
    print(f"USER QUERY: {query}")
    print("=" * 80)

    result = router.handle_query(query)

    # Some RouterAgentLLM implementations store logs; if not, we just print scenario + reply.
    logs = getattr(result, "logs", [])
    scenario = getattr(result, "scenario", "unknown")
    final_reply = getattr(result, "final_reply", result)

    print(f"\n[Detected scenario]: {scenario}\n")

    if logs:
        print("--- A2A LOGS ---")
        for line in logs:
            print(line)

    print("\n--- FINAL LLM RESPONSE ---")
    print(final_reply)

    extra = getattr(result, "extra", {})
    if extra:
        print("\n--- EXTRA DATA (from other agents) ---")
        print(extra)

    print("=" * 80)
    print("\n")


In [11]:
run_scenario("I need help with my account, customer ID 1")

USER QUERY: I need help with my account, customer ID 1
[customer-data-agent] Fetching customer: id=1

[Detected scenario]: task_allocation

--- A2A LOGS ---
[router-llm] classified: task_allocation
[router] → [data-agent]: fetch customer
[router] → [support-agent]: LLM account help

--- FINAL LLM RESPONSE ---
Hi John,

Thanks for reaching out! I see your account ID is 1, and I'm happy to help you with any issues you're experiencing.

Please let me know how I can assist you, and feel free to provide more details if needed.

Best regards,  
[Your Name]

--- EXTRA DATA (from other agents) ---
{'customer': {'id': 1, 'name': 'John Doe', 'email': 'updated@example.com', 'phone': '+1-555-0101', 'status': 'disabled', 'created_at': '2025-12-08 18:25:33', 'updated_at': '2025-12-09 05:09:31'}}




In [12]:
run_scenario("I want to cancel my subscription but I'm having billing issues, customer ID 1")

USER QUERY: I want to cancel my subscription but I'm having billing issues, customer ID 1
[customer-data-agent] Fetching customer history: id=1

[Detected scenario]: negotiation_escalation

--- A2A LOGS ---
[router-llm] classified: negotiation_escalation
[router] → [data-agent]: fetch customer history
[router] → [support-agent]: LLM escalation reasoning

--- FINAL LLM RESPONSE ---
Okay, John, thank you for reaching out. I see you're trying to cancel your subscription but are encountering a billing issue, and your customer ID is 1. Let's address this step by step.

## 1. Confirmation of Your Issue

First, I want to acknowledge that you have two things we need to handle:
*   You're looking to cancel your subscription.
*   You're experiencing a billing issue preventing or complicating that cancellation.

It sounds frustrating to have these two things happen together, especially since your account appears to have had some previous issues. We'll work to untangle this.

## 2. Investigating t

In [13]:
run_scenario("What's the status of all high-priority tickets for premium customers?")

USER QUERY: What's the status of all high-priority tickets for premium customers?
[customer-data-agent] Listing customers: status=active, limit=100
[customer-data-agent] Fetching customer history: id=2
[customer-data-agent] Fetching customer history: id=4
[customer-data-agent] Fetching customer history: id=5
[customer-data-agent] Fetching customer history: id=6
[customer-data-agent] Fetching customer history: id=7
[customer-data-agent] Fetching customer history: id=9
[customer-data-agent] Fetching customer history: id=10
[customer-data-agent] Fetching customer history: id=11
[customer-data-agent] Fetching customer history: id=12
[customer-data-agent] Fetching customer history: id=14
[customer-data-agent] Fetching customer history: id=15

[Detected scenario]: multi_step_coordination

--- A2A LOGS ---
[router-llm] classified: multi_step_coordination
[router] → [data-agent]: list premium customers
[data-agent] fetching history id=2
[data-agent] fetching history id=4
[data-agent] fetching 

## Summary

In this notebook we:

1. **Configured a local LLM backend** using Ollama + DeepSeek  
2. **Initialized three agents**:
   - `CustomerDataAgent` (tool / DB access via `mcp_tools.py`)
   - `SupportAgentLLM` (LLM-powered answer generation)
   - `RouterAgentLLM` (scenario classification + orchestration)
3. **Ran three multi-agent workflows**:
   - Task allocation: looking up a specific customer and answering a generic question
   - Negotiation / escalation: cancellation + billing conflict using customer history
   - Multi-step coordination: reporting high-priority tickets for premium customers

The router agent delegated work to the data agent and support agent, and the support agent used a local LLM (`deepseek-r1`) to generate natural language responses.

This demonstrates a complete **multi-agent, LLM-backed customer support system** that runs fully **locally** without external APIs.


In [9]:
run_scenario("Get customer information for ID 5")

USER QUERY: Get customer information for ID 5
[customer-data-agent] Fetching customer: id=5

[Detected scenario]: task_allocation

--- A2A LOGS ---
[router-llm] classified: task_allocation
[router] → [data-agent]: fetch customer
[router] → [support-agent]: LLM account help

--- FINAL LLM RESPONSE ---
Sure! Here's the customer information for ID 5:

*   **Name:** Charlie Brown
*   **Email:** charlie.brown@email.com
*   **Phone:** +1-555-0105
*   **Status:** active
*   **Created At:** 2025-12-08 18:25:33
*   **Updated At:** 2025-12-08 18:25:33

Let me know if you need anything else!

--- EXTRA DATA (from other agents) ---
{'customer': {'id': 5, 'name': 'Charlie Brown', 'email': 'charlie.brown@email.com', 'phone': '+1-555-0105', 'status': 'active', 'created_at': '2025-12-08 18:25:33', 'updated_at': '2025-12-08 18:25:33'}}




In [10]:
run_scenario("I'm customer 12345 and need help upgrading my account")

USER QUERY: I'm customer 12345 and need help upgrading my account
[customer-data-agent] Fetching customer: id=1

[Detected scenario]: task_allocation

--- A2A LOGS ---
[router-llm] classified: task_allocation
[router] → [data-agent]: fetch customer
[router] → [support-agent]: LLM account help

--- FINAL LLM RESPONSE ---
Sure, I'd be happy to help you with your account upgrade! Could you please confirm your customer ID and account status to ensure we're looking at the right information? This will help me assist you more effectively.

--- EXTRA DATA (from other agents) ---
{'customer': {'id': 1, 'name': 'John Doe', 'email': 'new@email.com', 'phone': '+1-555-0101', 'status': 'disabled', 'created_at': '2025-12-08 18:25:33', 'updated_at': '2025-12-10 06:14:05'}}




In [11]:
run_scenario("Show me all active customers who have open tickets")

USER QUERY: Show me all active customers who have open tickets
[customer-data-agent] Listing customers: status=active, limit=100
[customer-data-agent] Fetching customer history: id=2
[customer-data-agent] Fetching customer history: id=4
[customer-data-agent] Fetching customer history: id=5
[customer-data-agent] Fetching customer history: id=6
[customer-data-agent] Fetching customer history: id=7
[customer-data-agent] Fetching customer history: id=9
[customer-data-agent] Fetching customer history: id=10
[customer-data-agent] Fetching customer history: id=11
[customer-data-agent] Fetching customer history: id=12
[customer-data-agent] Fetching customer history: id=14
[customer-data-agent] Fetching customer history: id=15

[Detected scenario]: multi_step_coordination

--- A2A LOGS ---
[router-llm] classified: multi_step_coordination
[router] → [data-agent]: list active customers
[data-agent] fetching history id=2
[data-agent] fetching history id=4
[data-agent] fetching history id=5
[data-a

In [12]:
run_scenario("I've been charged twice, please refund immediately!")

USER QUERY: I've been charged twice, please refund immediately!
[customer-data-agent] Fetching customer history: id=1

[Detected scenario]: negotiation_escalation

--- A2A LOGS ---
[router-llm] classified: negotiation_escalation
[router] → [data-agent]: fetch customer history
[router] → [support-agent]: LLM escalation reasoning

--- FINAL LLM RESPONSE ---
Okay, John, I understand you're concerned about being charged twice. Let's address this.

**1. Confirmation of the Issue:**
You're right to point this out. We take billing accuracy very seriously, and it's concerning to hear about duplicate charges.

**2. Billing Investigation Steps:**
To resolve this, I will immediately start investigating your recent transactions. I'll:

*   Review your account's billing history for the last few days to identify the duplicate charge(s).
*   Check which service or product was associated with each charge.
*   Look into your current payment method and subscription status to understand how the duplicate

In [13]:
run_scenario("Update my email to new@email.com and show my ticket history")

USER QUERY: Update my email to new@email.com and show my ticket history
[customer-data-agent] Updating customer 1: {'email': 'new@email.com'}
[customer-data-agent] Fetching customer history: id=1

[Detected scenario]: multi_intent

--- A2A LOGS ---
[router-llm] classified: multi_intent
[router] multi-intent detected
[router] → [data-agent]: update email
[router] → [data-agent]: fetch history

--- FINAL LLM RESPONSE ---
Okay, here's the combined multi-intent support response:

---

**Subject: Email Update and Ticket History - new@email.com**

Hello,

Your email address has been successfully updated to `new@email.com`.

Here is your ticket history associated with this email address (customer ID: 1):

| Ticket ID | Issue | Status | Priority | Created At |
|---|---|---|---|---|
| 30 | Billing issue + cancellation request | open | high | 2025-12-09 09:12:28 |
| 29 | Billing issue + cancellation request | open | high | 2025-12-09 06:40:55 |
| 28 | pytest issue | open | high | 2025-12-09 05:0